In [1]:
import openai
import pandas as pd
import json
from IPython.display import Markdown
import sys

In [2]:
zz=openai.Model.list()

In [3]:
len(zz['data'])

62

In [4]:
[zz['data'][idx]['id'] for idx in range(len(zz['data'])) if 'gpt' in zz['data'][idx]['id']]

['gpt-4',
 'gpt-4-0613',
 'gpt-4-0314',
 'gpt-3.5-turbo-16k-0613',
 'gpt-3.5-turbo-16k',
 'gpt-3.5-turbo-0301',
 'gpt-3.5-turbo-0613',
 'gpt-3.5-turbo']

In [5]:
model_params = {
    'model': "gpt-4",
    'temperature': 0,
}

In [6]:
def print_save(string, file='../cmp2.md'):
    print(string)
    with open(file,'w') as f:
        f.write(string)

## Load `prompt_template.md`

In [7]:
sys.path.append('..')
from utils import *
from functools import partial

In [8]:
# prompt_dict=load_prompt_template('../prompt_template.md')
prompt_dict = partial(load_prompt_template,file='../prompt_template.md')

In [30]:
prompt_dict().keys()

dict_keys(['Preamble', 'Construct Kinetic Hamiltonian (continuum version)', 'Construct Kinetic Hamiltonian (lattice version)', 'Define each term in Kinetic Hamiltonian (continuum version)', 'Construct Potential Hamiltonian (continuum version)', 'Define each term in Potential Hamiltonian (continuum version)', 'Construct interaction Hamiltonian (real space, lattice version)', 'Construct interaction Hamiltonian (momentum space)', 'Convert from single-particle to second-quantized form, return in matrix', 'Convert from single-particle to second-quantized form, return in summation (expand the matrix)', 'Convert noninteracting Hamiltonian in real space to momentum space (continuum version)', 'Convert noninteracting Hamiltonian in real space to momentum space (lattice version)', 'Convert interacting Hamiltonian in real space to momentum space (lattice version)', 'Particle-hole transformation', 'Simplify the Hamiltonian in the particle-hole basis', "Wick's theorem", 'Extract quadratic term', 'H

In [9]:
print(prompt_dict['Construct Kinetic Hamiltonian (continuum version)'])

You will be instructed to describe the kinetic term of Hamiltonian in {system} in the {real|momentum} space in the {single-particle|second-quantized} form.   
The degrees of freedom of the system are: {dof}  
Express the Kinetic Hamiltonian {kinetic_symbol} using {var} which are only on the diagonal terms, and arrange the basis in the order of {order}.

Use the following conventions for the symbols:  
{def_var}




In [11]:
df=pd.DataFrame(list(prompt_dict.items()),columns=['Task','Prompt'])

## Construct Kinetic Hamiltonian (continuum version)

In [14]:
user_msg=[]
AI_msg=[]
sys_msg=[{'role': 'system', 'content': prompt_dict['Preamble']}]
kwargs_list=[]

In [84]:
kwargs = {
    'task':    'Construct Kinetic Hamiltonian (continuum version)',
    'system':    'the hole-doped AB-stacked MoTe2/WSe2',
    'real|momentum':    'real',
    'single-particle|second-quantized':    'single-particle',
    'dof':    'valley index (+K and -K valley), layer index (top and bottom layer)',
    'kinetic_symbol':    '$H_{Kinetic}(r)$',
    'var':    r'$E_{\tau,l}$',
    'order':    '(+K,bottom), (+K,top), (-K,bottom), (-K,top)',
    'def_var':
    r'''$l$ : layer index   
$t$ : top layer  
$b$ : bottom layer  
$\tau$ : valley index  
$+K$ : +K valley  
$-K$ : -K valley  
$k$ : momentum operator  
$E_{\tau,l}$ : energy dispersion for layer $l$ and valley $\tau$''',
    'answer':    '$H_{Kinetic}(r) = \\begin{pmatrix} E_{+K,b} & 0 & 0 & 0 \\\\ 0 & E_{+K,t} & 0 & 0 \\\\ 0 & 0 & E_{-K,b} & 0 \\\\ 0 & 0 & 0 & E_{-K,t} \\end{pmatrix}$'
}


In [85]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to describe the kinetic term of Hamiltonian in the hole-doped AB-stacked MoTe2/WSe2 in the real space in the single-particle form.   
The degrees of freedom of the system are: valley index (+K and -K valley), layer index (top and bottom layer).  
Express the Kinetic Hamiltonian $H_{Kinetic}(r)$ using $E_{\tau,l}$ which are only on the diagonal terms, and arrange the basis in the order of (+K,bottom), (+K,top), (-K,bottom), (-K,top).

Use the following conventions for the symbols:  
$l$ : layer index   
$t$ : top layer  
$b$ : bottom layer  
$\tau$ : valley index  
$+K$ : +K valley  
$-K$ : -K valley  
$k$ : momentum operator  
$E_{\tau,l}$ : energy dispersion for layer $l$ and valley $\tau$


In [12]:
user_msg.append(generate_prompt(kwargs,prompt_dict=prompt_dict))

In [13]:
messages=assembly_message(sys_msg,user_msg,AI_msg)

In [14]:
rs = openai.ChatCompletion.create(messages=messages, **model_params)

Markdown(rs['choices'][0]['message'].content)

Understood. The kinetic Hamiltonian $H_{Kinetic}(r)$ in real space for the hole-doped AB-stacked MoTe2/WSe2 system can be written as:

$H_{Kinetic}(r) = \begin{pmatrix} E_{+K,b} & 0 & 0 & 0 \\ 0 & E_{+K,t} & 0 & 0 \\ 0 & 0 & E_{-K,b} & 0 \\ 0 & 0 & 0 & E_{-K,t} \end{pmatrix}$

Here, the basis is arranged in the order of (+K,bottom), (+K,top), (-K,bottom), (-K,top). The diagonal terms represent the energy dispersion $E_{\tau,l}$ for each combination of valley index $\tau$ and layer index $l$.

In [17]:
AI_msg.append(dict(rs['choices'][0]['message']))

In [15]:
rs['choices'][0]['message'].content

'Understood. The kinetic Hamiltonian $H_{Kinetic}(r)$ in real space for the hole-doped AB-stacked MoTe2/WSe2 system can be written as:\n\n$H_{Kinetic}(r) = \\begin{pmatrix} E_{+K,b} & 0 & 0 & 0 \\\\ 0 & E_{+K,t} & 0 & 0 \\\\ 0 & 0 & E_{-K,b} & 0 \\\\ 0 & 0 & 0 & E_{-K,t} \\end{pmatrix}$\n\nHere, the basis is arranged in the order of (+K,bottom), (+K,top), (-K,bottom), (-K,top). The diagonal terms represent the energy dispersion $E_{\\tau,l}$ for each combination of valley index $\\tau$ and layer index $l$.'

In [16]:
kwargs_list.append(kwargs)

## Define each term in Kinetic Hamiltonian (continuum version)

In [ ]:
user_msg=[]
AI_msg=[]
sys_msg=[{'role': 'system', 'content': prompt_dict['Preamble']}]
kwargs_list=[]

In [86]:
kwargs = {
    'task': 'Define each term in Kinetic Hamiltonian (continuum version)',
    'var':r' four $E_{\tau,l}(k)$',
    'Description':r'''For all energy dispersions, $E_{l,\tau}(k)$, it characterizes the dispersion for free holes, namely, $E_{l,\tau}(k)= -\frac{\hbar^2 k^2}{2 m_l}$.   
In addition, for the top layer, $E_{\tau,t}(k)$, there is a shift of momentum $k$, to make the band bottom at $\tilde{\tau}\kappa$. You should infer the correct $\tilde{\tau}$.  
Finally, in the real space, the momentum $k=-i \partial_r$. You should keep the form of $k$ in the Hamiltonian for short notations but should remember $k$ is an operator.  ''',
    'expression_kinetic': '$H_{Kinetic}(r) = \\begin{pmatrix} E_{+K,b} & 0 & 0 & 0 \\\\ 0 & E_{+K,t} & 0 & 0 \\\\ 0 & 0 & E_{-K,b} & 0 \\\\ 0 & 0 & 0 & E_{-K,t} \\end{pmatrix}$',
    'kinetic_symbol': r'$H_{Kinetic}(r)$',
    'def_var':r'''$k$ : momentum operator  
$r$ : real space operator  
$m_l$ : the effective mass for layer $l$  
$\tilde{\tau}$ : takes value of +1 for $\tau=+K$ and -1 for $\tau=-K$  
$\pm\kappa$ : corners of monolayer moire Brillouin zone  ''',
'answer':'$H_{Kinetic}(r) = \\begin{pmatrix} -\\frac{\\hbar^2 k^2}{2 m_b} & 0 & 0 & 0 \\\\ 0 & -\\frac{\\hbar^2 (k-\\kappa)^2}{2 m_t} & 0 & 0 \\\\ 0 & 0 & -\\frac{\\hbar^2 k^2}{2 m_b} & 0 \\\\ 0 & 0 & 0 & -\\frac{\\hbar^2 (k+\\kappa)^2}{2 m_t} \\end{pmatrix}$'
}

In [87]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to construct each term in the matrix, namely  four $E_{\tau,l}(k)$.  
For all energy dispersions, $E_{l,\tau}(k)$, it characterizes the dispersion for free holes, namely, $E_{l,\tau}(k)= -\frac{\hbar^2 k^2}{2 m_l}$.   
In addition, for the top layer, $E_{\tau,t}(k)$, there is a shift of momentum $k$, to make the band bottom at $\tilde{\tau}\kappa$. You should infer the correct $\tilde{\tau}$.  
Finally, in the real space, the momentum $k=-i \partial_r$. You should keep the form of $k$ in the Hamiltonian for short notations but should remember $k$ is an operator.  
You should recall that $H_{Kinetic}(r) = \begin{pmatrix} E_{+K,b} & 0 & 0 & 0 \\ 0 & E_{+K,t} & 0 & 0 \\ 0 & 0 & E_{-K,b} & 0 \\ 0 & 0 & 0 & E_{-K,t} \end{pmatrix}$.   
Return the expression for  four $E_{\tau,l}(k)$ in the Kinetic Hamiltonian, and substitute it into the Kinetic Hamiltonian $H_{Kinetic}(r)$.  

Use the following conventions for the symbols (You should also obey the conventions in all my

In [13]:
user_msg.append(generate_prompt(kwargs,prompt_dict=prompt_dict))

In [14]:
user_msg

[{'role': 'user',
  'content': 'Now you will be instructed to construct each term in the matrix, namely  four $E_{\\tau,l}$.  \nFor all energy dispersions, $E_{l,\\tau}(k)$, it characterizes the dispersion for free holes, namely, $E_{l,\\tau}(k)= -\\frac{\\hbar^2 k^2}{2 m_l}$.   \nIn addition, for the top layer, $E_{\\tau,t}(k)$, there is a shift of momentum $k$, to make the band bottom at $\\tilde{\\tau}\\kappa$. You should infer the correct $\\tilde{\\tau}$.  \nFinally, in the real space, the momentum $k=-i \\partial_r$. You should keep the form of $k$ in the Hamiltonian for short notations but should remember $k$ is an operator.  \n\nYou should recall that $H_{Kinetic}(r) = \\begin{pmatrix} E_{+K,b} & 0 & 0 & 0 \\\\ 0 & E_{+K,t} & 0 & 0 \\\\ 0 & 0 & E_{-K,b} & 0 \\\\ 0 & 0 & 0 & E_{-K,t} \\end{pmatrix}$.  \nReturn the expression for  four $E_{\\tau,l}$ in the Kinetic Hamiltonian, and substitute it into the Kinetic Hamiltonian $H_{Kinetic}(r)$.  \n\nUse the following conventions for 

In [15]:
messages=assembly_message(sys_msg,user_msg,AI_msg)

In [16]:
rs = openai.ChatCompletion.create(messages=messages, **model_params)
Markdown(rs['choices'][0]['message'].content)

Understood. I will construct each term in the matrix, namely four $E_{\tau,l}$, based on your instructions.

The energy dispersion for free holes is given by $E_{l,\tau}(k)= -\frac{\hbar^2 k^2}{2 m_l}$.

For the top layer, there is a shift of momentum $k$ to make the band bottom at $\tilde{\tau}\kappa$. The correct $\tilde{\tau}$ should be +1 for $\tau=+K$ and -1 for $\tau=-K$.

In the real space, the momentum $k=-i \partial_r$. I will keep the form of $k$ in the Hamiltonian for short notations but remember $k$ is an operator.

Therefore, the expressions for the four $E_{\tau,l}$ in the Kinetic Hamiltonian are:

$E_{+K,b}(k) = -\frac{\hbar^2 k^2}{2 m_b}$

$E_{+K,t}(k) = -\frac{\hbar^2 (k-\kappa)^2}{2 m_t}$

$E_{-K,b}(k) = -\frac{\hbar^2 k^2}{2 m_b}$

$E_{-K,t}(k) = -\frac{\hbar^2 (k+\kappa)^2}{2 m_t}$

Substituting these into the Kinetic Hamiltonian $H_{Kinetic}(r)$, we get:

$H_{Kinetic}(r) = \begin{pmatrix} -\frac{\hbar^2 k^2}{2 m_b} & 0 & 0 & 0 \\ 0 & -\frac{\hbar^2 (k-\kappa)^2}{2 m_t} & 0 & 0 \\ 0 & 0 & -\frac{\hbar^2 k^2}{2 m_b} & 0 \\ 0 & 0 & 0 & -\frac{\hbar^2 (k+\kappa)^2}{2 m_t} \end{pmatrix}$.

In [32]:
AI_msg.append(dict(rs['choices'][0]['message']))

In [33]:
rs['choices'][0]['message'].content

'The energy dispersion for free holes in each layer and valley is given by $E_{l,\\tau}(k)= -\\frac{\\hbar^2 k^2}{2 m_l}$. For the top layer, the momentum $k$ is shifted to make the band bottom at $\\tilde{\\tau}\\kappa$, so $E_{\\tau,t}(k)= -\\frac{\\hbar^2 (k-\\tilde{\\tau}\\kappa)^2}{2 m_t}$.\n\nTherefore, the four energy dispersion terms are:\n\n$E_{+K,b}(k) = -\\frac{\\hbar^2 k^2}{2 m_b}$\n\n$E_{+K,t}(k) = -\\frac{\\hbar^2 (k-\\kappa)^2}{2 m_t}$\n\n$E_{-K,b}(k) = -\\frac{\\hbar^2 k^2}{2 m_b}$\n\n$E_{-K,t}(k) = -\\frac{\\hbar^2 (k+\\kappa)^2}{2 m_t}$\n\nSubstituting these into the kinetic Hamiltonian, we get:\n\n$H_{Kinetic}(r) = \\begin{pmatrix} -\\frac{\\hbar^2 k^2}{2 m_b} & 0 & 0 & 0 \\\\ 0 & -\\frac{\\hbar^2 (k-\\kappa)^2}{2 m_t} & 0 & 0 \\\\ 0 & 0 & -\\frac{\\hbar^2 k^2}{2 m_b} & 0 \\\\ 0 & 0 & 0 & -\\frac{\\hbar^2 (k+\\kappa)^2}{2 m_t} \\end{pmatrix}$\n\nRemember that $k$ is an operator, $k=-i \\partial_r$.'

In [34]:
kwargs_list.append(kwargs)

## Construct Potential Hamiltonian (continuum version)

In [ ]:
user_msg=[]
AI_msg=[]
sys_msg=[{'role': 'system', 'content': prompt_dict['Preamble']}]
kwargs_list=[]

In [88]:
kwargs = {
    'task':
    'Construct Potential Hamiltonian (continuum version)',
    'system':    'the hole-doped AB-stacked MoTe2/WSe2',
    'real|momentum':'real',
    'single-particle|second-quantized':'single-particle',
    'kinetic_symbol': r'$H_{Kinetic}(r)$',
    
    'Description': r'''At each valley, it is a 2 by 2 matrix $H_{Potential,\tau}(r)$, where each component is layer index. The diagonal term in this 2 by 2 matrix $H_{Potential,\tau}(r)$ is  intralayer tunneling $\Delta_l(r)$. The off-diagonal term in this 2 by 2 matrix $H_{Potential,\tau}(r)$ is interlayer potential $\Delta_{T,\tau}(r)$, which should be kept hermitian.  
There is no hybridization between $+K$ valley and $-K$ valley, i.e., the corresponding matrix elements in $H_{Potential}(r)$ are all zero''',
    'potential_symbol':'$H_{Potential}(r)$',
    'var':
    r'$\Delta_l(r)$ and $\Delta_{T,\tau}(r)$',
    'def_var': r'''  
$\Delta_l$: intralayer potential for layer index $l$  
$\Delta_{T,\tau}$ : interlayer tunneling at valley index $\tau$  
$H_{Potential}(r)$ : potential term of Hamiltonian in the real space in the single-particle form.  ''',
    'answer':'$H_{Potential}(r) = \\begin{pmatrix} \\Delta_b(r) & \\Delta_{T,+K}(r) & 0 & 0 \\\\ \\Delta_{T,+K}^*(r) & \\Delta_t(r) & 0 & 0 \\\\ 0 & 0 & \\Delta_b(r) & \\Delta_{T,-K}(r) \\\\ 0 & 0 & \\Delta_{T,-K}^*(r) & \\Delta_t(r) \\end{pmatrix}$'
    
}


In [89]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to describe the potential term of Hamiltonian $H_{Potential}(r)$ in the real space in the single-particle form.  
The potential Hamiltonian has the same degrees of freedom as the kinetic Hamiltonian $H_{Kinetic}(r)$.  
At each valley, it is a 2 by 2 matrix $H_{Potential,\tau}(r)$, where each component is layer index. The diagonal term in this 2 by 2 matrix $H_{Potential,\tau}(r)$ is  intralayer tunneling $\Delta_l(r)$. The off-diagonal term in this 2 by 2 matrix $H_{Potential,\tau}(r)$ is interlayer potential $\Delta_{T,\tau}(r)$, which should be kept hermitian.  
There is no hybridization between $+K$ valley and $-K$ valley, i.e., the corresponding matrix elements in $H_{Potential}(r)$ are all zero.  
Express the potential Hamiltonian $H_{Potential}(r)$ using $\Delta_l(r)$ and $\Delta_{T,\tau}(r)$.  

Use the following conventions for the symbols (You should also remember the conventions in my previous prompts if there are no conflicts. If you have conflicts in 

In [53]:
user_msg.append(generate_prompt(kwargs,prompt_dict=prompt_dict))

In [54]:
messages=assembly_message(sys_msg,user_msg,AI_msg)

In [55]:
messages

[{'role': 'system',
  'content': 'You are a physicist helping me to construct Hamiltonian and perform Hartree-Fock step by step based on my instructions. \nYou should follow the instruction strictly.\nYour reply should be succinct while complete. You should not expand any unwanted content.\nYou will be learning background knowledge by examples if necessary.\nConfirm and repeat your duty if you understand it.\n\n'},
 {'role': 'user',
  'content': 'You will be instructed to describe the potential term of Hamiltonian in the hole-doped AB-stacked MoTe2/WSe2 in the real space in the single-particle form.   \nThe degrees of freedom of the system are: valley index $\\tau$ (+K and -K valley), layer index $l$ (top and bottom layer)  \nAt each valley, it is a 2 by 2 matrix $H_{Potential,\\tau}(r)$, where each component is layer index. The diagonal term in this 2 by 2 matrix $H_{Potential,\\tau}(r)$ is  intralayer tunneling $\\Delta_l(r)$. The off-diagonal term in this 2 by 2 matrix $H_{Potential

In [56]:
rs = openai.ChatCompletion.create(messages=messages, **model_params)

Markdown(rs['choices'][0]['message'].content)

The potential Hamiltonian $H_{Potential}(r)$ can be expressed as a 4 by 4 matrix, with each 2 by 2 block corresponding to a valley index $\tau$. The diagonal elements of each 2 by 2 block represent the intralayer potential $\Delta_l(r)$, and the off-diagonal elements represent the interlayer potential $\Delta_{T,\tau}(r)$. The Hamiltonian can be written as:

$H_{Potential}(r) = \begin{pmatrix} 
\Delta_t(r) & \Delta_{T,+K}(r) & 0 & 0 \\
\Delta_{T,+K}^*(r) & \Delta_b(r) & 0 & 0 \\
0 & 0 & \Delta_t(r) & \Delta_{T,-K}(r) \\
0 & 0 & \Delta_{T,-K}^*(r) & \Delta_b(r) 
\end{pmatrix}$

Here, the first two rows and columns correspond to the +K valley, and the last two rows and columns correspond to the -K valley. The diagonal elements $\Delta_t(r)$ and $\Delta_b(r)$ represent the intralayer potential for the top and bottom layers, respectively. The off-diagonal elements $\Delta_{T,+K}(r)$ and $\Delta_{T,-K}(r)$ represent the interlayer potential for the +K and -K valleys, respectively. The elements outside the 2 by 2 blocks are all zero, indicating no hybridization between the +K and -K valleys.

In [41]:
AI_msg.append(dict(rs['choices'][0]['message']))

In [42]:
rs['choices'][0]['message'].content

'The potential Hamiltonian $H_{Potential}(r)$ in the real space for the hole-doped AB-stacked MoTe2/WSe2 system can be written as a 4x4 matrix, with the basis ordered as (+K,bottom), (+K,top), (-K,bottom), (-K,top). \n\nThe Hamiltonian is:\n\n$H_{Potential}(r) = \\begin{pmatrix} \\Delta_b(r) & \\Delta_{T,+K}(r) & 0 & 0 \\\\ \\Delta_{T,+K}^*(r) & \\Delta_t(r) & 0 & 0 \\\\ 0 & 0 & \\Delta_b(r) & \\Delta_{T,-K}(r) \\\\ 0 & 0 & \\Delta_{T,-K}^*(r) & \\Delta_t(r) \\end{pmatrix}$\n\nwhere $\\Delta_l(r)$ is the intralayer potential for layer $l$, and $\\Delta_{T,\\tau}(r)$ is the interlayer tunneling at valley index $\\tau$. The off-diagonal terms are zero because there is no hybridization between $+K$ valley and $-K$ valley. The interlayer tunneling terms are complex conjugates of each other to ensure the hermiticity of the Hamiltonian.'

In [43]:
kwargs_list.append(kwargs)

## Define each term in Potential Hamiltonian (continuum version)

In [24]:
user_msg=[]
AI_msg=[]
sys_msg=[{'role': 'system', 'content': prompt_dict['Preamble']}]
kwargs_list=[]

In [90]:
kwargs = {
    'task':
    'Define each term in Potential Hamiltonian (continuum version)',
    'potential_symbol':'$H_{Potential}(r)$',
    'var':    r'the intralayer potential $\Delta_l(r)$ and interlayer tunneling $\Delta_{T,\tau}(r)$',
    'real|momentum':'real',
    'single-particle|second-quantized':'single-particle',
    'kinetic_symbol':    '$H_{Kinetic}(r)$',
    'Description': r'''The intralayer on the top layer $\Delta_t(r)$ is $V_{zt}$.  
The intralayer on the bottom layer $\Delta_b(r)$ is $2V_b\sum_{j=1,3,5} \cos(g_j \cdot r  +\psi_b)$.  
The interlayer tunneling at valley $\tau$ is $\tilde{\tau}w(1+\omega^{\tilde{\tau}}e^{i\tilde{\tau} g_2 \cdot r} + \omega^{2\tilde{\tau}} e^{i\tilde{\tau} g_3 \cdot r})$. You should infer the correct $\tilde{\tau}$ such that $\tilde{\tau}$ does not appear in the answer.  ''',
    'def_var': r'''$V_b$ : bottom layer potential strength  
$\psi_b$ : bottom layer potential phase  
$g_j$ : reciprocal unit vector on the first shell corresponding in the moire Brillouin zone, j varies from 1 to 6.  
$r$ : real space position  
$\omega$ : phase factor to respect the threefold rotational symmetry, which is $e^{i2\pi/3}$, the hermitian conjugate is $\omega^{-1}=e^{-i2\pi/3}$  
$w$ : interlayer tunneling strength''',
    'expression_Potential': '$H_{Potential}(r) = \\begin{pmatrix} \\Delta_b(r) & \\Delta_{T,+K}(r) & 0 & 0 \\\\ \\Delta_{T,+K}^*(r) & \\Delta_t(r) & 0 & 0 \\\\ 0 & 0 & \\Delta_b(r) & \\Delta_{T,-K}(r) \\\\ 0 & 0 & \\Delta_{T,-K}^*(r) & \\Delta_t(r) \\end{pmatrix}$',
    'answer':'$H_{Potential}(r) = \\begin{pmatrix} 2V_b\\sum_{j=1,3,5} \\cos(g_j \\cdot r  +\\psi_b) & w(1+\\omega e^{i g_2 \\cdot r} + \\omega^2 e^{i g_3 \\cdot r}) & 0 & 0 \\\\ w(1+\\omega^{-1}e^{-i g_2 \\cdot r} + \\omega^{-2} e^{-i g_3 \\cdot r}) & V_{zt} & 0 & 0 \\\\ 0 & 0 & 2V_b\\sum_{j=1,3,5} \\cos(g_j \\cdot r  +\\psi_b) & -w(1+\\omega^{-1}e^{-i g_2 \\cdot r} + \\omega^{-2} e^{-i g_3 \\cdot r}) \\\\ 0 & 0 & -w(1+\\omega e^{i g_2 \\cdot r} + \\omega^2 e^{i g_3 \\cdot r}) & V_{zt} \\end{pmatrix}$'
    
}


In [91]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to construct each term in the matrix $H_{Potential}(r)$, namely, the intralayer potential $\Delta_l(r)$ and interlayer tunneling $\Delta_{T,\tau}(r)$.  
The intralayer on the top layer $\Delta_t(r)$ is $V_{zt}$.  
The intralayer on the bottom layer $\Delta_b(r)$ is $2V_b\sum_{j=1,3,5} \cos(g_j \cdot r  +\psi_b)$.  
The interlayer tunneling at valley $\tau$ is $\tilde{\tau}w(1+\omega^{\tilde{\tau}}e^{i\tilde{\tau} g_2 \cdot r} + \omega^{2\tilde{\tau}} e^{i\tilde{\tau} g_3 \cdot r})$. You should infer the correct $\tilde{\tau}$ such that $\tilde{\tau}$ does not appear in the answer.    
Return the expressions for the intralayer potential $\Delta_l(r)$ and interlayer tunneling $\Delta_{T,\tau}(r)$, and substitute it into the potential Hamiltonian $H_{Potential}(r)$.  

Use the following conventions for the symbols (You should also obey the conventions in all my previous prompts if you encounter undefined symbols. If you find it is never defined or have conflicts in 

In [27]:
user_msg.append(generate_prompt(kwargs,prompt_dict=prompt_dict))

In [28]:
messages=assembly_message(sys_msg,user_msg,AI_msg)

In [29]:
rs = openai.ChatCompletion.create(messages=messages, **model_params)

Markdown(rs['choices'][0]['message'].content)

The intralayer potential for the top layer $\Delta_t(r)$ is given by $V_{zt}$.

The intralayer potential for the bottom layer $\Delta_b(r)$ is given by $2V_b\sum_{j=1,3,5} \cos(g_j \cdot r  +\psi_b)$.

The interlayer tunneling at valley $+K$ is $\Delta_{T,+K}(r) = w (1+\omega e^{i g_2 \cdot r} + \omega^2 e^{i g_3 \cdot r})$.

The interlayer tunneling at valley $-K$ is $\Delta_{T,-K}(r) = w (1+\omega^{-1}e^{-i g_2 \cdot r} + \omega^{-2} e^{-i g_3 \cdot r})$.

Substituting these expressions into the potential Hamiltonian $H_{Potential}(r)$, we get:

$H_{Potential}(r) = \begin{pmatrix} 2V_b\sum_{j=1,3,5} \cos(g_j \cdot r  +\psi_b) & w (1+\omega e^{i g_2 \cdot r} + \omega^2 e^{i g_3 \cdot r}) & 0 & 0 \\ w (1+\omega^{-1}e^{-i g_2 \cdot r} + \omega^{-2} e^{-i g_3 \cdot r}) & V_{zt} & 0 & 0 \\ 0 & 0 & 2V_b\sum_{j=1,3,5} \cos(g_j \cdot r  +\psi_b) & w (1+\omega^{-1}e^{-i g_2 \cdot r} + \omega^{-2} e^{-i g_3 \cdot r}) \\ 0 & 0 & w (1+\omega e^{i g_2 \cdot r} + \omega^2 e^{i g_3 \cdot r}) & V_{zt} \end{pmatrix}$.

In [61]:
AI_msg.append(dict(rs['choices'][0]['message']))

In [62]:
rs['choices'][0]['message'].content

'The intralayer potential for the top layer is $\\Delta_t(r) = V_{zt}$.\n\nThe intralayer potential for the bottom layer is $\\Delta_b(r) = 2V_b\\sum_{j=1,3,5} \\cos(g_j \\cdot r  +\\psi_b)$.\n\nThe interlayer tunneling at valley $\\tau$ is $\\Delta_{T,\\tau}(r) = \\tau w(1+\\omega^{\\tau}e^{i\\tau g_2 \\cdot r} + \\omega^{2\\tau} e^{i\\tau g_3 \\cdot r})$.\n\nSubstituting these into the potential Hamiltonian, we get:\n\n$H_{Potential}(r) = \\begin{pmatrix} 2V_b\\sum_{j=1,3,5} \\cos(g_j \\cdot r  +\\psi_b) & w(1+\\omega e^{i g_2 \\cdot r} + \\omega^2 e^{i g_3 \\cdot r}) & 0 & 0 \\\\ w(1+\\omega^{-1}e^{-i g_2 \\cdot r} + \\omega^{-2} e^{-i g_3 \\cdot r}) & V_{zt} & 0 & 0 \\\\ 0 & 0 & 2V_b\\sum_{j=1,3,5} \\cos(g_j \\cdot r  +\\psi_b) & -w(1+\\omega e^{i g_2 \\cdot r} + \\omega^2 e^{i g_3 \\cdot r}) \\\\ 0 & 0 & -w(1+\\omega^{-1}e^{-i g_2 \\cdot r} + \\omega^{-2} e^{-i g_3 \\cdot r}) & V_{zt} \\end{pmatrix}$'

In [63]:
kwargs_list.append(kwargs)

## Convert from single-particle to second-quantized form, return in matrix

In [92]:
kwargs = {
    'task':'Convert from single-particle to second-quantized form, return in matrix',
    'real|momentum':'real',
    'nonint_symbol': r'$H^{0}(r)$',
    'potential_symbol':'$H_{Potential}(r)$',
    'kinetic_symbol':'$H_{Kinetic}(r)$',
    '$r$|$k$': '$r$',
    'second_nonint_symbol': r'$\hat{H}^{0}$',
    'def_var': r'''$\psi_{\tau,l}(r)$ : electron annihilation operator at valley index $\tau$ and layer index $l$  
$\vec{\psi}(r)$ : spinor containing all annihilation operators $\psi_{\tau,l}(r)$ following the same order of $H^{0}(r)$  
$H^{0}(r)$ : single-particle noninteracting Hamiltonian in the real space  
$\hat{H}^{0}$ : total noninteracting Hamiltonian in the second quantized form ''',
    'answer':''
}


In [93]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to construct the second quantized form of the total noninteracting Hamiltonian in the real space.  
The noninteracting Hamiltonian in the real space $H^{0}(r)$ is the sum of Kinetic Hamiltonian $H_{Kinetic}(r)$ and Potential Hamiltonian $H_{Potential}(r)$.  
To construct the second quantized form of a Hamiltonian. You should construct the creation and annihilation operators from the basis explicitly. You should follow the EXAMPLE below to convert a Hamiltonian from the single-particle form to second-quantized form.  
Finally by "total", it means you need to take a summation over the real space position $r$.   
Return the second quantized form of the total noninteracting Hamiltonian $\hat{H}^{0}$  

Use the following conventions for the symbols (You should also obey the conventions in all my previous prompts if you encounter undefined symbols. If you find it is never defined or has conflicts in the conventions, you should stop and let me know):  
$\psi_{\tau,l}(r)

In [75]:
user_msg.append(generate_prompt(kwargs,prompt_dict=prompt_dict))

In [76]:
messages=assembly_message(sys_msg,user_msg,AI_msg)

In [77]:
rs = openai.ChatCompletion.create(messages=messages, **model_params)

Markdown(rs['choices'][0]['message'].content)

InvalidRequestError: This model's maximum context length is 8192 tokens. However, your messages resulted in 9798 tokens. Please reduce the length of the messages.

In [ ]:
AI_msg.append(dict(rs['choices'][0]['message']))

In [ ]:
rs['choices'][0]['message'].content

'The intralayer potential for the top layer is $\\Delta_t(r) = V_{zt}$.\n\nThe intralayer potential for the bottom layer is $\\Delta_b(r) = 2V_b\\sum_{j=1,3,5} \\cos(g_j \\cdot r  +\\psi_b)$.\n\nThe interlayer tunneling at valley $\\tau$ is $\\Delta_{T,\\tau}(r) = \\tau w(1+\\omega^{\\tau}e^{i\\tau g_2 \\cdot r} + \\omega^{2\\tau} e^{i\\tau g_3 \\cdot r})$.\n\nSubstituting these into the potential Hamiltonian, we get:\n\n$H_{Potential}(r) = \\begin{pmatrix} 2V_b\\sum_{j=1,3,5} \\cos(g_j \\cdot r  +\\psi_b) & w(1+\\omega e^{i g_2 \\cdot r} + \\omega^2 e^{i g_3 \\cdot r}) & 0 & 0 \\\\ w(1+\\omega^{-1}e^{-i g_2 \\cdot r} + \\omega^{-2} e^{-i g_3 \\cdot r}) & V_{zt} & 0 & 0 \\\\ 0 & 0 & 2V_b\\sum_{j=1,3,5} \\cos(g_j \\cdot r  +\\psi_b) & -w(1+\\omega e^{i g_2 \\cdot r} + \\omega^2 e^{i g_3 \\cdot r}) \\\\ 0 & 0 & -w(1+\\omega^{-1}e^{-i g_2 \\cdot r} + \\omega^{-2} e^{-i g_3 \\cdot r}) & V_{zt} \\end{pmatrix}$'

In [ ]:
kwargs_list.append(kwargs)

## Convert from single-particle to second-quantized form, return in summation (expand the matrix)

In [94]:
kwargs = {
    'task':'Convert from single-particle to second-quantized form, return in summation (expand the matrix)',
    'second_nonint_symbol':  r'$\hat{H}^{0}$',
    'matrix_element_symbol': r'$H^{0}_{\tau_1,l_1,\tau_2,l_2}(r)$',
    'basis_symbol': r'$\psi_{\tau,l}(r)$',
    'def_var': r'''$H^{0}_{\tau_1,l_1,\tau_2,l_2}(r)$ : the matrix element of single-particle noninteracting Hamiltonian in the real space  ''',
    'answer':''
}


In [95]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to expand the second-quantized form Hamiltonian $\hat{H}^{0}$ using $H^{0}_{\tau_1,l_1,\tau_2,l_2}(r)$ and $\psi_{\tau,l}(r)$. You should follow the EXAMPLE below to expand the Hamiltonian.  
You should use any previous knowledge to simplify it. For example, if any term of $H^{0}_{\tau_1,l_1,\tau_2,l_2}(r)$ is zero, you should remove it from the summation.  
Return the expanded form of $\hat{H}^{0}$ after simplication.  

Use the following conventions for the symbols (You should also obey the conventions in all my previous prompts if you encounter undefined symbols. If you find it is never defined or has conflicts in the conventions, you should stop and let me know):  
$H^{0}_{\tau_1,l_1,\tau_2,l_2}(r)$ : the matrix element of single-particle noninteracting Hamiltonian in the real space  

===  
EXAMPLE:  
For a $\hat{H}=\vec{\psi}^\dagger H \vec{\psi}$, where $\vec{\psi}=\begin{pmatrix} \psi_a \\ \psi_b \end{pmatrix}$ and $\vec{\psi}^\dagger=\begin{pmatrix} \psi

## Convert noninteracting Hamiltonian in real space to momentum space (continuum version)

In [105]:
kwargs = {
    'task':'Convert noninteracting Hamiltonian in real space to momentum space (continuum version)',
    'real_creation_op': r'$\psi_{\tau,l}^\dagger(r)$',
    'momentum_creation_op': r'$c_{\tau,l}^\dagger(k)$',
    'def_FT': r'$c_{\tau,l}^\dagger(k)= \frac{1}{\sqrt{V}} \int dr \psi_{\tau,l}^\dagger(r) e^{i k \cdot r}$',
    'real_var': '$r$',
    'entire_real|fBZ': 'entire real space',
    'second_nonint_symbol':  r'$\hat{H}^{0}$',
    'def_var': r'''$c_{l,\tau}(k)$ : electron annihilation operator at momentum $k$, layer $l$, and valley $\tau$  
$c_{l,\tau}^\dagger(k)$ : electron creation operator at momentum $k$, layer $l$, and valley $\tau$  
$V$ : area of unit cell in the real space  ''',
    'answer':''
}


In [106]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to convert the total noninteracting Hamiltonian in the second quantized form from the basis in real space to the basis by momentum space.  
To do that, you should apply the Fourier transformation to $\psi_{\tau,l}^\dagger(r)$ in the real space to the $c_{\tau,l}^\dagger(k)$ in the momentum space, which is defined as $c_{\tau,l}^\dagger(k)= \frac{1}{\sqrt{V}} \int dr \psi_{\tau,l}^\dagger(r) e^{i k \cdot r}$, where $r$ is integrated over the entire real space. You should follow the EXAMPLE below to apply the Fourier transformation.  
Express the total noninteracting Hamiltonian $\hat{H}^{0}$ in terms of $c_{\tau,l}^\dagger(k)$. Simplify any summation index if possible.  

Use the following conventions for the symbols (You should also obey the conventions in all my previous prompts if you encounter undefined symbols. If you find it is never defined or has conflicts in the conventions, you should stop and let me know):  
$c_{l,\tau}(k)$ : electron annihilation opera

## Particle-hole transformation

In [96]:
kwargs = {
    'task':'Particle-hole transformation',
    'hole_op': r'$b_{l,\tau}(k)$',
    'particle_op': r'$c_{l,\tau}^\dagger(k)$',
    'particle_creation_op': r'$c_{l,\tau}$',
    'hole_creation_op': r'$b_{l,\tau}(k)^\dagger$',
    'particle_annihilation_op': r'$c_{l,\tau}^\dagger$',
    'hole_annihilation_op': r'$b_{l,\tau}(k)$',
    'second_nonint_symbol':  r'$\hat{H}^{0}$',

    'def_var': r'''$b_{l,\tau}(k)$ : hole annihilation operator at momentum $k$, layer $l$, and valley $\tau$''',
    'answer':''
}


In [97]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to perform a particle-hole transformation.  
Define a hole operator, $b_{l,\tau}(k)$, which equals $c_{l,\tau}^\dagger(k)$.  
You should replace $c_{l,\tau}$ with $b_{l,\tau}(k)^\dagger$, and $c_{l,\tau}^\dagger$ with $b_{l,\tau}(k)$. You should follow the EXAMPLE below to apply the particle-hole transformation.
Return the $\hat{H}^{0}$ in the hole operators.

Use the following conventions for the symbols (You should also obey the conventions in all my previous prompts if you encounter undefined symbols. If you find it is never defined or has conflicts in the conventions, you should stop and let me know):  
$b_{l,\tau}(k)$ : hole annihilation operator at momentum $k$, layer $l$, and valley $\tau$

===  
EXAMPLE:  
Give a Hamiltonian  $\hat{H}=\sum_{k_1,k_2} c^\dagger(k_1) h(k_1,k_2) c(k_2)$ , and the particle-hole transformation as $b(k)=c^\dagger(k)$. The transformed Hamiltonian is $\hat{H}=\sum_{k_1,k_2} b(k_1) h(k_1,k_2) b^\dagger(k_2)$


## Simplify the Hamiltonian in the particle-hole basis

In [102]:
kwargs = {
    'task':'Simplify the Hamiltonian in the particle-hole basis',
    'second_nonint_symbol':  r'$\hat{H}^{0}$',
    'hole_op': r'$b_{l,\tau}(k)$',
    'index_1': r'$l_1$/$k_1$',
    'index_2': r'$l_2$/$k_2$',
    'op': r'$b$',
    'Ham_op': r'$H$',
    'def_var': r'''$\delta_{i,j}$ : Kronecker delta function, take the value of 1 if $i=j$, and 0 otherwise.  
$`[b_i, b_j]_{+}`$ : anticommuting relation, i.e., $`[b_i, b_j]_+= b_i b_j +b_j b_i`$  
$\hat{H}^{0,final}$ : the final version of noninteracting Hamiltonian $\hat{H}^{0}$ in the hole basis after simplification.''',
    'answer':''
}


In [104]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to simplify the $\hat{H}^{0}$ in the hole basis.  
You should use canonical commutator relation for fermions to reorder the hole operator to the normal order. Normal order means that creation operators always appear before the annihilation operators.  You should follow the EXAMPLE below to simplify it to the normal order.  
Express the $\hat{H}^{0}$ in the normal order of $b_{l,\tau}(k)$ and also make $l_1$/$k_1$ always appear before $l_2$/$k_2$ in the index of $b$ and $H$.  

Use the following conventions for the symbols (You should also obey the conventions in all my previous prompts if you encounter undefined symbols. If you find it is never defined or has conflicts in the conventions, you should stop and let me know):  
$\delta_{i,j}$ : Kronecker delta function, take the value of 1 if $i=j$, and 0 otherwise.  
$`[b_i, b_j]_{+}`$ : anticommuting relation, i.e., $`[b_i, b_j]_+= b_i b_j +b_j b_i`$  
$\hat{H}^{0,final}$ : the final version of noninteracting Hamil

## Construct interaction Hamiltonian (momentum space)

In [109]:
kwargs = {
    'task':'Construct interaction Hamiltonian (momentum space)',
    'second_int_symbol':  r'$\hat{H}^{int}$',
    'index': r'valley index $\tau_i$ and layer index $l_i$',
    'momentum': r'momentum $k_i$',
    '$k_i$|$k_i$ and $b_i$': r'$k_i$',
    'interaction': r'the dual-gate-screened Coulomb interaction',
    'int_form': r'$2\pi e^2\tanh(|q| d)/(\epsilon |q|)$',
    'normalization_factor': r'$\frac{1}{2 V}$',
    'op': r'$b_{l,\tau}(k)$',
    'def_var': r'''$d$ : distance between dual gates  
$\epsilon$ : dielectric constant  
$V(q)$ : dual-gate-screened Coulomb interaction in the momentum space''',
    'answer':''
}


In [110]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to construct the interaction part of the Hamiltonian $\hat{H}^{int}$ in the momentum space.  
The interaction Hamiltonian is a product of four parts.
The first part is the product of four operators with two creation and two annihilation operators following the normal order, namely, creation operators are before annihilation operators. You should follow the order of $1,2,2,1$ for the valley index $\tau_i$ and layer index $l_i$, and $1,2,3,4$ for the momentum $k_i$. 
The second part is the constraint of total momentum conservation, namely the total momentum of all creation operators should be the same as that of all annihilation operators. For each operator, the momentum is $k_i$.  
The third part is the interaction form. You should use the dual-gate-screened Coulomb interaction with $V(q)=$2\pi e^2\tanh(|q| d)/(\epsilon |q|)$$, where $q$ is the transferred total momentum between a creation operator and an annilation operator with the same valley index $\tau_i$ and

## Wick's theorem

In [119]:
kwargs = {
    'task':"Wick's theorem",
    'second_int_symbol':  r'$\hat{H}^{int}$',
    'HF_symbol': r'$\hat{H}^{int,HF}$',
    'def_var': r'''$\langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_2) \rangle$ : expectation of $b_{l_1,\tau_1}^\dagger(k_1) b_{l_2,\tau_2}(k_2)$ over the many-body ground states.  
$\hat{H}^{int,HF}$ : Interaction term after Hartree-Fock approximation''',
    'answer':''
}


In [121]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to perform a Hartree-Fock approximation to expand the interaction term $\hat{H}^{int}$.  
You should use Wick's theorem to expand the four-fermion term in $\hat{H}^{int}$ into quadratic terms. You should strictly follow the EXAMPLE below to expand using Wick's theorem, select the correct EXAMPLE by noticing the order of four term product with and without ${}^dagger$, and be extremely cautious about the order of the index and sign before each term.  
You should only preserve the normal terms. Here, the normal terms mean the product of a creation operator and an annihilation operator.  
Return the expanded interaction term after Hartree-Fock approximation $\hat{H}^{int,HF}$.

Use the following conventions for the symbols (You should also obey the conventions in all my previous prompts if you encounter undefined symbols. If you find it is never defined or has conflicts in the conventions, you should stop and let me know):  
$\langle b_{l_1,\tau_1}^\dagger(k_1) b_{l_

## Extract quadratic term

In [123]:
kwargs = {
    'task':"Extract quadratic term",
    'HF_symbol': r'$\hat{H}^{int,HF}$',
    'bilinear_op': r'$b^\dagger b$',
    'HF_2_symbol': r'$\hat{H}^{int,HF,2}$',
    'def_var': r'''$\hat{H}^{int,HF,2}$ : the quadratic terms in $\hat{H}^{int,HF}$
''',
    'answer':''
}


In [124]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to extract the quadratic terms in the $\hat{H}^{int,HF}$.  
The quadratic terms mean terms that are proportional to $b^\dagger b$, which excludes terms that are solely expectations or products of expectations.  
You should only preserve the quadratic terms in $\hat{H}^{int,HF}$, which is called $\hat{H}^{int,HF,2}$.
Return this Hamiltonian $\hat{H}^{int,HF,2}$.  

Use the following conventions for the symbols (You should also obey the conventions in all my previous prompts if you encounter undefined symbols. If you find it is never defined or has conflicts in the conventions, you should stop and let me know):  
$\hat{H}^{int,HF,2}$ : the quadratic terms in $\hat{H}^{int,HF}$


## Swap the index to combine Hartree and Fock terms

In [ ]:
kwargs = {
    'task':"Swap the index to combine Hartree and Fock terms",
    'HF_2_symbol': r'$\hat{H}^{int,HF,2}$',
    'expval': r'$\langle b_{\tau_1,l_1}^\dagger(k_1) b_{\tau_1,l_1}(k_4) \rangle$',

    
    'HF_symbol': r'$\hat{H}^{int,HF}$',
    'bilinear_op': r'$b^\dagger b$',
    'def_var': r'''$\hat{H}^{int,HF,2}$ : the quadratic terms in $\hat{H}^{int,HF}$
''',
    'answer':''
}


In [ ]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to extract the quadratic terms in the $\hat{H}^{int,HF}$.  
The quadratic terms mean terms that are proportional to $b^\dagger b$, which excludes terms that are solely expectations or products of expectations.  
You should only preserve the quadratic terms in $\hat{H}^{int,HF}$, which is called $\hat{H}^{int,HF,2}$.
Return this Hamiltonian $\hat{H}^{int,HF,2}$.  

Use the following conventions for the symbols (You should also obey the conventions in all my previous prompts if you encounter undefined symbols. If you find it is never defined or has conflicts in the conventions, you should stop and let me know):  
$\hat{H}^{int,HF,2}$ : the quadratic terms in $\hat{H}^{int,HF}$


In [425]:
--

SyntaxError: invalid syntax (3659366440.py, line 1)

## Save jsonl

In [66]:
with open('2111.01152.jsonl', 'w') as f:
    for dict_obj in kwargs_list:
        f.write(json.dumps(dict_obj) + '\n')

# Draft

In [ ]:
with open('1106.6060/1106.6060.jsonl', 'w') as f:
    json.dump(kwargs,f)

In [ ]:
with open('cmp2.md','w') as f:
    f.write(generate_prompt(kwargs)['content'])

In [26]:
preamble=r"""You are a physicist helping me to construct Hamiltonian and perform Hartree-Fock step by step based on my instructions. 
You should follow the instruction strictly.
Your reply should be succinct while complete. You should not expand any unwanted content.
You will be learning background knowledge by examples if necessary.
Confirm and repeat your duty if you understand it."""

In [20]:
template=r"""You will be instructed to describe the kinetic term of Hamiltonian in the hole-doped AB-stacked MoTe2/WSe2 in the real space in the single-particle form.   
The degrees of freedom of the system are: valley index (+K and -K valley), layer index (top and bottom layer). 
Express the Kinetic Hamiltonian $H_{Kinetic}(r)$ using $E_{\tau,l}$ which are only on the diagonal terms, and arrange the basis in the order of (+K,bottom), (+K,top), (-K,bottom), (-K,top).

Use the following conventions for the symbols:  
$l$ : layer index   
$t$ : top layer  
$b$ : bottom layer  
$\tau$ : valley index  
$+K$ : +K valley  
$-K$ : -K valley  
$k$ : momentum operator  
$E_{l,\tau}$ : energy dispersion for layer $l$ and valley $\tau$   """

In [21]:
rs=openai.ChatCompletion.create(model="gpt-4", temperature=0,messages=[{'role':'system','content':preamble},{'role':'user', 'content':template}])

In [22]:
Markdown(rs['choices'][0]['message'].content)

Understood. The kinetic Hamiltonian $H_{Kinetic}(r)$ in the real space for the hole-doped AB-stacked MoTe2/WSe2 system can be written in the single-particle form as follows:

$H_{Kinetic}(r) = \begin{pmatrix} E_{b,+K} & 0 & 0 & 0 \\ 0 & E_{t,+K} & 0 & 0 \\ 0 & 0 & E_{b,-K} & 0 \\ 0 & 0 & 0 & E_{t,-K} \end{pmatrix}$

Here, the basis is arranged in the order of (+K,bottom), (+K,top), (-K,bottom), (-K,top). The diagonal terms represent the energy dispersion for each combination of layer and valley.

In [10]:
template=r'''Describe the kinetic term of free fermion Hamiltonian in the AB-stacked MoTe$_2$/WSe$_2$. 
This system has a spin-valley locking, so you should construct a continuum Hamiltonian which contains two valleys.
Each valley is a 2 by 2 matrix with each component for the top or bottom layer.
The kinetic term is hole-like free fermion dispersion for both layers, with a minus sign before. 
There is a momentum shift on the top layer to make the band bottom at the corner of the moire Brillouin zone.
Return a 4 by 4 matrix, where the basis is (+K, bottom), (+K, top), (-K, bottom), (-K, top). +K and -K are two valleys in the corners of monolayer moire Brillouin zone.


Use the following conventions for the symbols:
$H_\tau$ : Valley-dependent free Hamiltonian 
$\tau$ : valley index
$m_b$ ($m_t$) : effective mass on the bottom (top) layer
$\kappa$ : corner of moire Brillouin Zone
$ r $ : real space position
$ k $ : momentum operator
'''

In [11]:
print(template)

Describe the kinetic term of free fermion Hamiltonian in the AB-stacked MoTe$_2$/WSe$_2$. 
This system has a spin-valley locking, so you should construct a continuum Hamiltonian which contains two valleys.
Each valley is a 2 by 2 matrix with each component for the top or bottom layer.
The kinetic term is hole-like free fermion dispersion for both layers, with a minus sign before. 
There is a momentum shift on the top layer to make the band bottom at the corner of the moire Brillouin zone.
Return a 4 by 4 matrix, where the basis is (+K, bottom), (+K, top), (-K, bottom), (-K, top). +K and -K are two valleys in the corners of monolayer moire Brillouin zone.


Use the following conventions for the symbols:
$H_\tau$ : Valley-dependent free Hamiltonian 
$\tau$ : valley index
$m_b$ ($m_t$) : effective mass on the bottom (top) layer
$\kappa$ : corner of moire Brillouin Zone
$ r $ : real space position
$ k $ : momentum operator



In [48]:
x=0
print(f' {x} {{}} {{{x}}}')

 0 {} {0}
